In [1]:
#reading the file
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from sklearn.preprocessing import LabelEncoder
import string
from tqdm import tqdm
tqdm.pandas()
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Dense, Dropout, GlobalAvgPool1D
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

In [2]:
loc = 'data/IMDB_Dataset.csv'
df = pd.read_csv(loc)

In [3]:
nlp = spacy.load('en_core_web_sm')

In [6]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [4]:
# 1 for positve Sentiment
le = LabelEncoder()
df['sentiment'] = le.fit_transform(df['sentiment'])

df['review'] = df['review'].apply(lambda x:x.replace('<br /><br />', ''))
df['review'] = df['review'].apply(lambda x:x.replace('..', ''))
df['review'] = df['review'].apply(lambda x:x.replace('...', ''))
df['review'] = df['review'].apply(lambda x:x.replace('....', ''))

In [11]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to manyAryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and moreso scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't

In [5]:
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [11]:
def tokenizer_f(x):
    sent = nlp(x)
    token = []
    for i in (sent):
        if i.lemma_ == '-PRON-':
            token.append(i.lower_)
        elif not i.is_stop and not i.lemma_.lower() in punct:
            token.append(i.lemma_.lower())          
    return ' '.join(token)
            

In [29]:
df['review'] = df['review'].progress_apply(tokenizer_f)

100%|██████████| 50000/50000 [33:40<00:00, 24.75it/s]


In [33]:
X = df['review'].to_numpy()
y = df['sentiment'].values

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify = y)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((33500,), (16500,), (33500,), (16500,))

In [39]:
tokenizer = Tokenizer(num_words=10000,oov_token="<00v>")

tokenizer.fit_on_texts(X_train)

In [40]:
X_train.shape

(33500,)

In [41]:
X_train[0]

"sure it making good film storyline good bit bland acting good i understand olivia d'abo pronounce australian accent her character suppose raise my big problem wardrobe i know rule average american consider frumpy dresser self respect european anna colour combination positively ghastly potato sack like sad excuse coat she wear film me break hive i suppose idea realistic possible school teacher walk prada simple mean absolute lack taste word wise"

In [42]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [45]:
X_train = pad_sequences(X_train, maxlen=400)
X_test = pad_sequences(X_test, maxlen=400)

In [48]:
X_train.shape

(33500, 400)

In [50]:
model = Sequential()
model.add(Embedding(input_dim = 10000, output_dim = 15,input_length = 400))
model.add(GlobalAvgPool1D())
model.add(Dense(15, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.compile(loss = "sparse_categorical_crossentropy", optimizer = 'adam', metrics = ['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=2)

Train on 33500 samples, validate on 16500 samples
Epoch 1/10
33500/33500 - 16s - loss: 0.4446 - accuracy: 0.7903 - val_loss: 0.2860 - val_accuracy: 0.8873
Epoch 2/10
33500/33500 - 14s - loss: 0.2444 - accuracy: 0.9043 - val_loss: 0.2823 - val_accuracy: 0.8860
Epoch 3/10
33500/33500 - 14s - loss: 0.2040 - accuracy: 0.9219 - val_loss: 0.2893 - val_accuracy: 0.8861
Epoch 4/10
33500/33500 - 15s - loss: 0.1783 - accuracy: 0.9327 - val_loss: 0.3131 - val_accuracy: 0.8792
Epoch 5/10
33500/33500 - 14s - loss: 0.1621 - accuracy: 0.9404 - val_loss: 0.3543 - val_accuracy: 0.8661
Epoch 6/10
33500/33500 - 14s - loss: 0.1490 - accuracy: 0.9456 - val_loss: 0.3935 - val_accuracy: 0.8645
Epoch 7/10
33500/33500 - 14s - loss: 0.1360 - accuracy: 0.9506 - val_loss: 0.3373 - val_accuracy: 0.8835
Epoch 8/10
33500/33500 - 14s - loss: 0.1263 - accuracy: 0.9563 - val_loss: 0.3570 - val_accuracy: 0.8830
Epoch 9/10
33500/33500 - 14s - loss: 0.1194 - accuracy: 0.9582 - val_loss: 0.3752 - val_accuracy: 0.8819
Epoch

In [52]:
pred = model.predict(X_test)

In [63]:
roc_auc_score(y_test,pred[:,1])

0.9442910486685032

In [64]:
accuracy_score(y_test, pred[:,1]>0.5)

0.8792727272727273

In [65]:
confusion_matrix(y_test, pred[:,1]>0.5)

array([[7261,  989],
       [1003, 7247]])